# ASMSA: Prepare and check input files

**Next steps**
- [tune.ipynb](tune.ipynb): Perform initial hyperparameter tuning for this molecule
- [train.ipynb](train.ipynb): Use results of previous tuning in more thorough training
- [md.ipynb](md.ipynb): Use a trained model in MD simulation with Gromacs

In [ ]:
%cd trpcage

In [ ]:
#avoid TF to consume GPU memory
import tensorflow as tf

tf.config.set_visible_devices([], 'GPU')
tf.config.list_logical_devices()

import torch

In [ ]:
#avoid TF to consume GPU memory
import tensorflow as tf

tf.config.set_visible_devices([], 'GPU')
tf.config.list_logical_devices()

import torch

In [ ]:
import mdtraj as md
import numpy as np
import urllib.request
import asmsa
import os
import re
import tensorflow as tf
import gromacs as gmx
import gromacs.fileformats as gf
import matplotlib.pyplot as plt
import numpy as np

## Prepare input files

Tryptophan cage files are downloaded in this section from our Google drive. 

This is for demonstration purpose, in real use the inputs should be placed here, and _conf, traj, topol, index_ variables set to their filenames names.

In [ ]:
# Define input files
base = 'trpcage'

# input conformation, it should not contain hydrogens
conf = base + '_correct.pdb'

# input trajectory
# atom numbering must be consistent with {conf}, no hydrogens as well

traj = 'trpcage_red.xtc'

# everything else is generated with pdb2gmx to make sure the files are consistent

#### Density of additional internal coordinates

In how many randomly sampled distances from all atom-to-atom one atom should appear in average

In [ ]:
nb_density = 2 # integer in [1, n_atoms-1]

In [ ]:
topol = base + '.top'
index = base + '.ndx'
gro = base + '.gro'

with open('inputs.py','w') as i:
    i.write(f'''
base = '{base}'
conf = '{conf}'
traj = '{traj}'
topol = '{topol}'
index = '{index}'
gro = '{gro}'

nb_density = {nb_density}
''')

In [ ]:
gmx.pdb2gmx(f=conf, ignh=True,p=topol,n=index,o=gro,water='tip3p',ff='amber99sb-ildn')

## Sanity checks

In [ ]:
# Load the trajectory, it should report expected numbers of frames and atoms/residua

tr = md.load(traj,top=conf)
idx=tr[0].top.select("name CA")

# for trivial cases like Ala-Ala, where superposing on CAs fails
#idx=tr[0].top.select("element != H")

tr.superpose(tr[0],atom_indices=idx)

In [ ]:
# Visual check, all frames should look "reasonable"

# Because of different conventions of numbering atoms in proteins,
# PDB file {conf} and the trajectory {traj} can become inconsistent, and this would appear here 
# as rather weird shapes of the molecule

import nglview as nv

v = nv.show_mdtraj(tr)
v.clear()
v.add_representation("licorice")
#v.add_representation('ball+stick', selection='ZN2', radius=0.5, color="green") no Zn :(
v

## Split datasets

Split trajectory into 3 parts. Each part will represent training, validation and testing dataset respectively. The workflow is following:
1. Shuffle configurations in trajectory
2. Select proportions to divide the trajectory
3. Divide the trajectory
4. Compute RMSD between
   * **train x validation** trajectory and filter similar structures in train trajectory
   * **train x test** trajectory and filter similar structures in train trajectory
   * **test x validation** trajectory and filter similar structures in test trajectory
5. Transform into internal coordinates
6. Save internal coordinates as datasets which can be loaded in **train.ipynb** and **tune.ipynb** notebooks

In [ ]:
# shuffle the trajectory so the configurations are dispersed across all datasets
np.random.shuffle(tr.xyz)

In [ ]:
# - set proportions for train, validation and test datasets
# - proportions must be equal to 1 when added together
train = .7
validation = .15
test = .15

assert train + validation + test == .9999999999999999 or 1

tr_i = len(tr) * train
X_train = tr.slice(slice(0,int(tr_i)))

va_i = len(tr) * validation
X_validate = tr.slice(slice(int(tr_i),int(tr_i)+int(va_i)))

te_i = len(tr) * test
X_test = tr.slice(slice(int(tr_i)+int(va_i),len(tr)))

X_train.xyz.shape, X_validate.xyz.shape, X_test.xyz.shape

In [ ]:
X_train.save_xtc('train.xtc')
X_validate.save_xtc('validate.xtc')
X_test.save_xtc('test.xtc')

## Recovery

In [ ]:
# eventual recovery

X_train = md.load_xtc('train.xtc',conf)
X_validate = md.load_xtc('validate.xtc',conf)
X_test = md.load_xtc('test.xtc',conf)


# Keep on calculate

In [ ]:
# get RMSD from train trajectory compared to validation trajectory
gmx.select(s=conf,on='backbone.ndx',select='Backbone')

In [ ]:
gmx.rms(s=conf,f='train.xtc',f2='validate.xtc',n='backbone.ndx',m='trainxval_rmsd.xpm')

In [ ]:
# load the RMDS matrix
txv = gf.XPM('trainxval_rmsd.xpm')
txv.array.shape

In [ ]:
# minima per row -- for each configuration in train, how far is the nearest one from validation
txv_min = np.min(txv.array,axis=1)
txv_min.shape

In [ ]:
plt.hist(txv_min,bins=50)
plt.show()

In [ ]:
# drop similar structures (to validation trajectory) in train trajectory to avoid dataset being biased
txv_difference = 0.05

train_tr = X_train[np.argwhere(txv_min > txv_difference).flat]
train_tr.xyz.shape

In [ ]:
train_tr.save_xtc('tmp_train.xtc')
gmx.rms(s=conf,f='tmp_train.xtc',f2='test.xtc',n='backbone.ndx',m='trainxtest_rmsd.xpm')

In [ ]:
txt = gf.XPM('trainxtest_rmsd.xpm')
txt.array.shape

In [ ]:
txt_min = np.min(txt.array,axis=1)
txt_min.shape

In [ ]:
plt.hist(txt_min,bins=50)
plt.show()

In [ ]:
# ... one more time with test trajectory & test x validation...
txt_difference = 0.05

x_train = train_tr[np.argwhere(txt_min > txt_difference).flat]
x_train.save_xtc('x_train.xtc')

In [ ]:
# test x validation
gmx.rms(f='test.xtc',f2='validate.xtc',s=conf,n='backbone.ndx',m='testxvalidate_rmsd.xpm')

In [ ]:
txv = gf.XPM('testxvalidate_rmsd.xpm')
txv.array.shape

In [ ]:
txv_min = np.min(txv.array,axis=1)
txv_min.shape

In [ ]:
plt.hist(txv_min,bins=50)
plt.show()

In [ ]:
# ... one more time with test trajectory & test x validation...
txv_difference = 0.05

x_test = X_test[np.argwhere(txv_min > txv_difference).flat]
x_test.save_xtc('x_test.xtc')

In [ ]:
# skip thorough RMS
! ln train.xtc x_train.xtc
! ln test.xtc x_test.xtc

# Recovery

In [ ]:
exec(open('inputs.py').read())

In [ ]:
# recovery

x_train = md.load('x_train.xtc', top=conf)
x_test = md.load('x_test.xtc', top=conf)


# Anyway

In [ ]:
# get shapes of filtered trajectories that are to be used as datasets
validate_tr = md.load('validate.xtc', top=conf)

trajs = [x_train, validate_tr, x_test]
x_train.xyz.shape, validate_tr.xyz.shape, x_test.xyz.shape

In [ ]:
# reshuffle the geometries to get frame last so that we can use vectorized calculations later on
geoms = [ np.moveaxis(t.xyz,0,-1) for t in trajs]
print ([ g.shape for g in geoms ])

In [ ]:
# save geometries

tf.data.Dataset.from_tensor_slices(geoms[0]).save('datasets/geoms/train')
tf.data.Dataset.from_tensor_slices(geoms[1]).save('datasets/geoms/validate')
tf.data.Dataset.from_tensor_slices(geoms[2]).save('datasets/geoms/test')

# Internal coordinates computation

Exercise the ASMSA library on your input. Just check everything appears to work.

There are multiple options that can be combined:
- use traditional internal coordinates (bond distances, angles, and dihedrals) or not
- include additional distances between atoms that may not be bound to express protein folding state more directly
   - dense (all-to-all) atom distances, feasible for very small peptides only
   - sparse atom distances (only some pairs are chosen)
   

We save the computed internal coordinates for training, and a feature extraction model here, therefore everything in the other notebooks should work too.


## New - Traditional internal coordinates (all bond distances, angles, and torsions)


In [ ]:
'''
# mol = asmsa.Molecule(conf,topol)

# internal coordinates and sparse any-any atom distances (not restricted to bonds)
# eventually, top (and index) can be left out to use sparse distances only

sparse_dists = asmsa.NBDistancesSparse(geoms[0].shape[0], density=nb_density)
mol=asmsa.Molecule(pdb=conf,top=topol,ndx=index,fms=[sparse_dists])

# dense distances are feasible for very small (upto 5 residua) peptides only

# dense_dists = asmsa.NBDistancesDense(geom.shape[0])
# mol = asmsa.Molecule(pdb=conf,top=topol,ndx=index,fms=[dense_dists])
'''

In [ ]:
#print(+ mol.__dict__)

##### Alternative: only backbone + Cbeta anlges and dihedrals

In [ ]:
with open('backbone.ndx') as i:
    i.readline()
    bb = np.array([ int(j)-1 for j in " ".join(i.readlines()).split() ])

In [ ]:
bb

In [ ]:
# backbone angles and dihedrals
#angles = np.array([ bb[i:i+3] for i in range(0,len(bb)-3) ])
diheds = np.array([ bb[i:i+4] for i in range(0,len(bb)-4) ])
diheds

In [ ]:
# XXX: select alpha carbons and matching betas
tr1 = md.load(conf)
cas = tr1.topology.select('name CA and not resname GLY')
cbs = tr1.topology.select('name CB')
assert(len(cas) == len(cbs))

In [ ]:
cas

In [ ]:
# indices of CAs (non-GLY) on the backbone
cai = np.argwhere(bb.reshape(1,-1) == cas.reshape(-1,1))[:,1]
cai

In [ ]:
# angles of CB-CA-X, where X is the next atom on the backbone
cbangles = np.array([[ cbs[0], cas[0], bb[cai[0]+1] ]] +
                   [[cbs[i], bb[cai[i]], bb[cai[i]-1] ] for i in range(1,len(cbs))])
# just check 
cbangles+1

In [ ]:
cbdiheds = np.array([[ cbs[0], cas[0], bb[cai[0]+1], bb[cai[0]+2] ]] +
                   [[cbs[i], bb[cai[i]], bb[cai[i]-1], bb[cai[i]-2]] for i in range(1,len(cbs))])
cbdiheds+1

In [ ]:
'''
# just angles
mol=asmsa.Molecule(pdb=conf,n_atoms=geoms[0].shape[0],
                   angles=np.concatenate((angles,cbangles)),
                   diheds=np.concatenate((diheds,cbdiheds)))
'''

In [ ]:
'''
# molecule model with explicit angles and dihedrals, and sparse distances from among Calpha and Cbetas
# (don't bother with distances now)
sparse_dists = asmsa.NBDistancesSparse(geoms[0].shape[0], density=nb_density, atoms = np.concatenate((cas,cbs)))
mol=asmsa.Molecule(pdb=conf,n_atoms=geoms[0].shape[0],
                   angles=np.concatenate((angles,cbangles)),
                   diheds=np.concatenate((diheds,cbdiheds)),
                   fms=[sparse_dists]) 
'''

In [ ]:
sparse_dists = asmsa.NBDistancesSparse(geoms[0].shape[0], density=nb_density, atoms = np.concatenate((cas,cbs)))
dense_dists = asmsa.NBDistancesDense(geoms[0].shape[0])


mol=asmsa.Molecule(pdb=conf,n_atoms=geoms[0].shape[0],
                   diheds=np.concatenate((diheds,cbdiheds)),
                   fms=[sparse_dists]) 

In [ ]:
mol_model = mol.get_model()

example_input = torch.randn((*geoms[0].shape[:2],1))

In [ ]:
mol_model.angles

In [ ]:
mol_model.get_indices()

In [ ]:
len(sparse_dists.bonds)

In [ ]:
len(dense_dists.bonds)

##### Save the features (molecule) model

In [ ]:
mol_model = mol.get_model()

example_input = torch.randn((*geoms[0].shape[:2],1))
traced_script_module = torch.jit.trace(mol_model, example_input)

traced_script_module.save('features.pt')

##### Compute the interanal coordinates now

In [ ]:
intcoords = [ mol.intcoord(g).T for g in geoms]
print(
    [ g.shape for g in geoms ],
    [ i.shape for i in intcoords ]
)

In [ ]:
[train,validate,test] = intcoords

In [ ]:
geoms[2].shape

In [ ]:
d = mol_model.dihed4_model(torch.from_numpy(geoms[2])).numpy()

In [ ]:
d.shape,test.shape

In [ ]:
def forward(self, geoms):
#        geoms = input.reshape(self.n_atoms, 3, -1)
    a12 = geoms[self.atoms[:, 1]] - geoms[self.atoms[:, 0]]
    a23 = geoms[self.atoms[:, 2]] - geoms[self.atoms[:, 1]]
    a34 = geoms[self.atoms[:, 3]] - geoms[self.atoms[:, 2]]

#        a12 = torch.nn.functional.normalize(a12, p=2, dim=1)
#        a23 = torch.nn.functional.normalize(a23, p=2, dim=1)
#        a34 = torch.nn.functional.normalize(a34, p=2, dim=1)

    vp1 = torch.nn.functional.normalize(torch.cross(a12,a23,axis=1))
    vp2 = torch.nn.functional.normalize(torch.cross(a23,a34,axis=1))
    vp3 = torch.nn.functional.normalize(torch.cross(vp1,a23,axis=1))

    sp1 = torch.sum(vp1 * vp2, axis=1)
    sp2 = torch.sum(vp3 * vp2, axis=1)

    """ original:
    # output for i-th dihedral angle
        aa = np.arctan2(sp1,sp2) - np.pi * .5
        return np.sin(aa), np.cos(aa)
    """

    #NOTE: Why adding two variables that determine each other? It the angle better?
    # return torch.nn.functional.normalize(torch.stack([-sp2, sp1]), p=2, dim=0).reshape(2*len(self.atoms), geoms.shape[2])
    return sp1,sp2
    #return torch.stack([-sp2, sp1]).reshape(2*len(self.atoms), geoms.shape[2])


In [ ]:
sp1,sp2 = forward(mol_model.dihed4_model,torch.from_numpy(geoms[2]))

In [ ]:
sp1.shape,sp2.shape

In [ ]:
torch.stack([-sp2, sp1]).reshape((146,-1)).shape

In [ ]:
a1 = torch.from_numpy(np.array([[1,2,3],[4,5,6]]))
a1

In [ ]:
a1.shape

In [ ]:
a2 = a1+.2
a2

In [ ]:
torch.stack([a1,a2]).reshape(4,-1)

In [ ]:
test_from_model.shape

In [ ]:
# validate the saved model -- should yield nearly 0.

test_from_model = mol_model(torch.from_numpy(geoms[2])).numpy()
np.max(test - test_from_model.T)

In [ ]:
# normalize training set
train_mean = np.mean(train,axis=0)
train -= train_mean
train_scale = np.std(train,axis=0)
train /= train_scale

In [ ]:
# normalize test and validation sets
test -= train_mean
test /= train_scale
validate -= train_mean
validate /= train_scale

## Old gold

In [ ]:

# mol = asmsa.Molecule(conf,topol)

# internal coordinates and sparse any-any atom distances (not restricted to bonds)
# eventually, top (and index) can be left out to use sparse distances only

sparse_dists = asmsa.NBDistancesSparse(geoms[0].shape[0], density=nb_density)
mol=asmsa.Molecule(pdb=conf,top=topol,ndx=index,fms=[sparse_dists])

# dense distances are feasible for very small (upto 5 residua) peptides only

# dense_dists = asmsa.NBDistancesDense(geom.shape[0])
# mol = asmsa.Molecule(pdb=conf,top=topol,ndx=index,fms=[dense_dists])

#### Alternative: only backbone + Cbeta anlges and dihedrals

In [ ]:
with open('backbone.ndx') as i:
    i.readline()
    bb = np.array([ int(j)-1 for j in " ".join(i.readlines()).split() ])

In [ ]:
bb

In [ ]:
# backbone angles and dihedrals
angles = np.array([ bb[i:i+3] for i in range(0,len(bb)-3) ])
diheds = np.array([ bb[i:i+4] for i in range(0,len(bb)-4) ])
angles, diheds

In [ ]:
# XXX: select alpha carbons and matching betas
tr1 = md.load(conf)
cas = tr1.topology.select('name CA and not resname GLY')
cbs = tr1.topology.select('name CB')
assert(len(cas) == len(cbs))

In [ ]:
cas

In [ ]:
# indices of CAs (non-GLY) on the backbone
cai = np.argwhere(bb.reshape(1,-1) == cas.reshape(-1,1))[:,1]
cai

In [ ]:
# angles of CB-CA-X, where X is the next atom on the backbone
cbangles = np.array([[ cbs[0], cas[0], bb[cai[0]+1] ]] +
                   [[cbs[i], bb[cai[i]], bb[cai[i]-1] ] for i in range(1,len(cbs))])
# just check 
cbangles+1

In [ ]:
cbdiheds = np.array([[ cbs[0], cas[0], bb[cai[0]+1], bb[cai[0]+2] ]] +
                   [[cbs[i], bb[cai[i]], bb[cai[i]-1], bb[cai[i]-2]] for i in range(1,len(cbs))])
cbdiheds+1

In [ ]:
# just angles
mol=asmsa.Molecule(pdb=conf,n_atoms=geoms[0].shape[0],
                   angles=np.concatenate((angles,cbangles)),
                   diheds=np.concatenate((diheds,cbdiheds)))

In [ ]:
# molecule model with explicit angles and dihedrals, and sparse distances from among Calpha and Cbetas
# (don't bother with distances now)
sparse_dists = asmsa.NBDistancesSparse(geoms[0].shape[0], density=nb_density, atoms = np.concatenate((cas,cbs)))
mol=asmsa.Molecule(pdb=conf,n_atoms=geoms[0].shape[0],
                   angles=np.concatenate((angles,cbangles)),
                   diheds=np.concatenate((diheds,cbdiheds)),
                   fms=[sparse_dists]) 

In [ ]:
mol_model = mol.get_model()

example_input = torch.randn((*geoms[0].shape[:2],1))

In [ ]:
mol_model.angles

In [ ]:
mol_model.get_indices()

In [ ]:
sparse_dists.bonds

#### Save the features (molecule) model

In [ ]:
mol_model = mol.get_model()

example_input = torch.randn((*geoms[0].shape[:2],1))
traced_script_module = torch.jit.trace(mol_model, example_input)

traced_script_module.save('features.pt')

#### Compute the interanal coordinates now

In [ ]:
intcoords = [ mol.intcoord(g).T for g in geoms]
print(
    [ g.shape for g in geoms ],
    [ i.shape for i in intcoords ]
)

In [ ]:
[train,validate,test] = intcoords

In [ ]:
# validate the saved model -- should yield nearly 0.

test_from_model = mol_model(torch.from_numpy(geoms[2])).numpy()
np.max(test - test_from_model.T)

In [ ]:
# normalize training set
train_mean = np.mean(train,axis=0)
train -= train_mean
train_scale = np.std(train,axis=0)
train /= train_scale

In [ ]:
# normalize test and validation sets
test -= train_mean
test /= train_scale
validate -= train_mean
validate /= train_scale

## Save 

In [ ]:
# save for usage in tune/train/test phase

tf.data.Dataset.from_tensor_slices(train).save('datasets/intcoords/train')
tf.data.Dataset.from_tensor_slices(validate).save('datasets/intcoords/validate')
tf.data.Dataset.from_tensor_slices(test).save('datasets/intcoords/test')

np.savetxt('datasets/intcoords/mean.txt',train_mean)
np.savetxt('datasets/intcoords/scale.txt',train_scale)

# Density of the conformational space

- Sample the training trajectory randomly
- For each point in the trajectory:
  - calculate RMSD to all points in the sample
  - pick some number $n$ of nearest ones
  - calculate the _density_ at this point as $$ d = \sum_{i=1}^n e^{-d_i} / n $$  i.e. the nearer the sample points are, the higher the density
 
Altogether, $d$ roughly corresponds to the probability that the molecule during simulation ends up in this area of the conformational space.

In [ ]:
sample_size = 5000
x_train = md.load('x_train.xtc', top=conf)
tr_sample = x_train[np.random.choice(len(x_train),sample_size,False)]
tr_sample.save('sample.xtc')

In [ ]:
gmx.rms(f='x_train.xtc',f2='sample.xtc',s=conf,n='backbone.ndx',m='sample_rmsd.xpm')

In [ ]:
rms = gf.XPM('sample_rmsd.xpm')

#### Visual check to verify the sample size is representative
- typically, not many distances should be less than 0.1 nm and more than 1 nm 
(the latter depends on the molecule, can be more for e.g. big disordered proteins)
- the histogram should be semi-smooth

In [ ]:
plt.hist(rms.array.flatten(),bins=50)
plt.show()

In [ ]:
k_nearest = 200
rms_sort = np.sort(rms.array.astype(np.float32))
erms = np.exp(-rms_sort[:,:k_nearest])
dens = (np.sum(erms,axis=1)-1.) / (erms.shape[1] - 1)

#### Histogram of densities
- quite high number of points should fall above 0.8, those are low energy basins
- the interval [0.5, 1.0] should be reasonably covered
- on the contrary, too many points below 0.4 would indicate either insufficient sampling above or too sparse trajectory

In [ ]:
plt.hist(dens,bins=20)
plt.show()

In [ ]:
len(dens),len(x_train)

In [ ]:
np.savetxt('datasets/train_density.txt',dens)